[View in Colaboratory](https://colab.research.google.com/github/shuvamg007/draw/blob/master/prof_match_2.ipynb)

In [1]:
# INIT (Colab only) 

!pip install gensim && wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.zip && unzip wiki.en.zip
!pip install clarifai

    100% |████████████████████████████████| 23.6MB 697kB/s 
    100% |████████████████████████████████| 1.4MB 14.7MB/s 
    100% |████████████████████████████████| 133kB 28.6MB/s 
    100% |████████████████████████████████| 4.7MB 6.8MB/s 
    100% |████████████████████████████████| 61kB 25.3MB/s 
    100% |████████████████████████████████| 552kB 27.3MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
--2018-10-16 07:25:01--  https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.20.37
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.20.37|:443... connected

In [0]:
# Imports

from gensim import models
from pandas import DataFrame, read_csv
import http.client, urllib.parse, json
from clarifai.rest import ClarifaiApp
import pandas as pd
import os
import csv
import requests

In [0]:
# Load Fasttext

model = models.KeyedVectors.load_word2vec_format('wiki.en.vec', limit=500000)

In [0]:
# test

from google.colab import files

def upload():
    uploaded = files.upload()
 
def download(path):
    files.download(path)

In [54]:
upload()

Saving advertisement.json to advertisement (1).json


In [6]:
# test

print(model.most_similar('entertainment'))
print(model.similarity('coke', 'caffeine'))

[('intertainment', 0.8752332925796509), ('entertainmentwise', 0.7589191198348999), ('entertainments', 0.7339411973953247), ('zintertainment', 0.7227294445037842), ('entertainment/wwe', 0.7135158777236938), ('entertainmentnow', 0.6949946284294128), ('entertainment/', 0.6818501949310303), ('tainment', 0.6627265214920044), ('edutainment', 0.6331461668014526), ('//entertainment', 0.6006413698196411)]
0.3742156


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [8]:
# test visualize

from sklearn.manifold import TSNE
from matplotlib import pylab

words = [word for word in model.index2word[:1000]]
embeddings = [model[word] for word in words]
words_embedded = TSNE(n_components=2).fit_transform(embeddings)

pylab.figure(figsize=(20, 20))
for i, label in enumerate(words):
  x, y = words_embedded[i, :]
  pylab.scatter(x, y)
  pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                 ha='right', va='bottom')
pylab.show()

KeyboardInterrupt: ignored

In [0]:
# caption analyzer

def caption_filter(caption, result):
  text_analytics_base_url = "https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.0/"
  subscription_key = "77708ea602414c869e56865d03e42b90"
  headers   = {"Ocp-Apim-Subscription-Key": subscription_key}
  sentiment_api_url = text_analytics_base_url + "sentiment"
  key_phrase_api_url = text_analytics_base_url + "keyPhrases"

  # spell check
  url_spell = "https://api.cognitive.microsoft.com/bing/v7.0/spellcheck"
  payload = "mode=proof&mkt=en-US&text="
  headers_spell = {
      'ocp-apim-subscription-key': "968f4f044b284badaa0c8aba079c3ac1",
      'content-type': "application/x-www-form-urlencoded"
      }


  try:          
    if len(caption) > 0:
      response = requests.post(url_spell, data=payload+caption, headers=headers_spell)
      corrections = response.json()["flaggedTokens"]
      for correction in corrections:
        try:
          incorrect = correction["token"]
          correct = correction["suggestions"][0]["suggestion"]
          caption.replace(incorrect, correct)
        except:
          continue
          
      print("Caption - " + caption)
      documents = { 'documents': [{ 'id': '1', 'text': caption }] }

      # not being considered
      response  = requests.post(sentiment_api_url, headers=headers, json=documents)
      sentiments = response.json()
      print("Sentiment - " + str(sentiments['documents'][0]['score']))

      response  = requests.post(key_phrase_api_url, headers=headers, json=documents)
      key_phrase = response.json()
      print("Key Phrase - " + str(key_phrase["documents"][0]["keyPhrases"]))

      for phrase in key_phrase['documents'][0]['keyPhrases']:
        if phrase not in result:
          result[phrase] = [1, 1]
        else:
          result[phrase][0] += 1
          result[phrase][1] += 1

  except Exception as e:
    print("error_caption_filter")


In [0]:
# interests analyzer

def interest_filter(interests, result):
  for interest in interests:
    if interest["isMain"] == "True":
      result[interest["interestName"].lower()] = [100, 1]
    else:
      result[interest["interestName"].lower()] = [50, 1]

In [0]:
# image tagging

def image_tag(url, result):
  
  try:    
    # Immaga results
    api_key = 'acc_2eeecb0fb721a8a'
    api_secret = 'af186bc0677bdcfccd3900a15557f46a'

    response = requests.get('https://api.imagga.com/v1/tagging?url=%s' % url, auth=(api_key, api_secret))
    data = response.json()
    tags = data["results"][0]["tags"]

    for tag in tags:      
      if tag["tag"] not in result:
        result[tag["tag"]] = [tag["confidence"], 1]
      else:
        result[tag["tag"]][0] += tag["confidence"]
        result[tag["tag"]][1] += 1

      
    # Clarifai results
    app = ClarifaiApp(api_key='87408d50d73b4697b11911f7f048e071')
    model = app.public_models.general_model
    model.model_version = 'aa7f35c01e0642fda5cf400f543e7c40'

    tags = model.predict_by_url(url=url)
    tags = tags["outputs"][0]["data"]["concepts"]
    
    for tag in tags:      
      if tag["name"] not in result:
        result[tag["name"]] = [tag["value"]*100, 1]
      else:
        result[tag["name"]][0] += tag["value"]*100
        result[tag["name"]][1] += 1


    # Microsoft API results (need new keys)
    response = requests.post("https://westcentralus.api.cognitive.microsoft.com/vision/v2.0/analyze?visualFeatures=Description,Tags&subscription-key=4c618df1e194495d97554a0d10efe78e", headers={"content-type":"application/json"}, json={"url": url})

    data = response.json()
#     print(data)
    tags = data["tags"]
    
    for tag in tags:
      if tag["name"] not in result:
        result[tag["name"]] = [tag["confidence"]*100, 1]
      else:
        result[tag["name"]][0] += tag["confidence"]*100
        result[tag["name"]][1] += 1
    
  except Exception as e:
    print("error_image_tagging - " + e)


In [0]:
# similarity calculator
# TODO
def calc_sim(result, advrt):
  idx = 1
  ad_score = []
  for key, res in result.items():
    key_score = []
    for token in advrt:
      try:
        key_score.append(model.similarity(key, token) * (res[0]/res[1]))
      except:
        key_score.append(0)
    ad_score.append(key_score)

  return ad_score

In [66]:
# main

user_url = "userData.json"
advert_url = "ads.json"

with open(user_url) as f:
    data = json.load(f)
    
with open(advert_url) as f:
    advts = json.load(f)

# response = requests.get(url)
# data = response.json()
print(data)

userScore = []
userId = data["email"]
addresses = data["userAddresses"]  # need to see use case

interests = data["interests"]
orders = data["orders"]

contacts = data["contacts"]  # can use interests

count = 1

results = {}
interest_filter(interests, results)

for order in orders:
  photos = order["photos"]
  for photo in photos:
    url = photo["photoUrl"]
    caption = photo["caption"]
    
    image_tag(url, results)
    caption_filter(caption, results)
    
    print(results)
    
#     idx = 1
#     img_score = []
#     for result in results:
#       if model[result[0]] != 0:
#         img_score = ((img_score * (idx-1)) + model[result[0]]) / idx
#         idx += 1
      
#     userScore = ((userScore * (count-1)) + img_score) / count
    
# print(userScore)

adScore = []
for advt in advts:
  adScore.append(calc_sim(results, advt["keywords"]))
  
# print(adScore)

{'numberOfPhostcards': 77, 'profilePicPath': None, 'profilePicThumbnailPath': None, 'name': 'Demo', 'phoneNumber': '5558882220', 'countryCode': '+91', 'dob': '10/10/1994', 'sex': 'Male', 'email': 'demo@gmail.com', 'userId': '60cecc3e-a008-4174-a7ad-d1bda6ea0cc8', 'status': 'success', 'userAddresses': [{'pincode': '305001', 'houseNo': 'flat', 'locality': 'nagpur', 'landmark': None, 'addressType': 'HOME', 'city': 'Ajmer', 'state': 'Rajasthan', 'id': '3fa16457-fc53-4e88-a924-f907a55e8528', 'contactId': None, 'userId': '60cecc3e-a008-4174-a7ad-d1bda6ea0cc8', 'isPrimary': False, 'status': 'success'}, {'pincode': '400101', 'houseNo': 'hash', 'locality': 'wgegbev', 'landmark': '', 'addressType': 'HOME', 'city': 'Mumbai', 'state': 'Maharashtra', 'id': '8ac032c3-88bf-4a76-9b00-e2620c85102c', 'contactId': None, 'userId': '60cecc3e-a008-4174-a7ad-d1bda6ea0cc8', 'isPrimary': True, 'status': 'success'}, {'pincode': '400101', 'houseNo': 'how', 'locality': 'shdhbdbd', 'landmark': '', 'addressType': '

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [67]:
scores = []
print(len(adScore[0][0]))

for score in adScore:
  scores.append(sum(map(sum,score)))
  
print(scores)
  
print(advts[scores.index(max(scores))]["name"])

5
[4437.52374566133, 4678.579950154855]
Navneet


In [27]:
print(len(results))

120


In [63]:
ls

ads.json            sample_data/   wiki.en.bin  wiki.en.zip
advertisement.json  userData.json  wiki.en.vec
